In [1]:
import pickle

LABEL_COL = -1 

with open('raw_all_plays_1645.pickle', 'rb') as f:
    raw = pickle.load(f)

with open('X_all_plays_1645.pickle', 'rb') as f:
    X = pickle.load(f)
X.shape



(42192, 1645)

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import numpy as np
np.set_printoptions(suppress=True)

import time
import math
###############
# training ####
###############

n = len(X)*.25
print(n)
train_val_split_delim = math.floor(n * .8)
val_test_split_delim = math.floor(n * .9)
end_delim = int(n)


print("total rows: {} train val split delim: {} val test split delim: {}".format(len(X), train_val_split_delim, val_test_split_delim))
# numpy note: addressing [:delim] is exclusive of delim, but
#                         [delim:] is inclusive of delim


X_train = X[0:train_val_split_delim]
Y_train = raw[0:train_val_split_delim,LABEL_COL]

X_val = X[train_val_split_delim:val_test_split_delim]
Y_val = raw[train_val_split_delim:val_test_split_delim,LABEL_COL]

X_test = X[val_test_split_delim:end_delim]
Y_test = raw[val_test_split_delim:end_delim,LABEL_COL]

if len(X_test) != len(Y_test):
    raise ValueError("mismatched arrays! {} {}".format(len(preds), len(y_true)))

    
# Utility function to report best scores (from sklearn example documentation)
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            
            
param_grid = {"alpha": [.25,.5,1,1.5,2]}#
mdl = MultinomialNB(alpha=1.5)
mdl.fit(X_train, Y_train)
# grid_search = GridSearchCV(mdl, param_grid=param_grid)
# grid_search.fit(X_train, Y_train)
# report(grid_search.cv_results_)


10548.0
total rows: 42192 train val split delim: 8438 val test split delim: 9493


MultinomialNB(alpha=1.5, class_prior=None, fit_prior=True)

In [13]:
# def persist_features(X, name='X_all_plays.pickle'):
#     with open(name, 'wb') as f:
#         pickle.dump(X, f, pickle.HIGHEST_PROTOCOL)

#persist_features(mdl, 'mdl_all_plays_pos_7_gram.pickle')
# persist_features(X, 'X_all_plays_pos_gram.pickle')

In [14]:
# evaluation ###
################
#second class is original


#y_true = raw[train_val_split_delim:val_test_split_delim,LABEL_COL]
val_preds = mdl.predict(X_val)

#y_true = [int(raw[int(idx)][-1]) for idx in val_indices]

# brier = brier_score_loss(y_true, preds[:,0]) 
# print("brier score:", brier)

#currently slightly worse than randomly permuting preds

def accuracy(preds, y_true):
    if len(preds) != len(y_true):
        raise ValueError("mismatched arrays! {} {}".format(len(preds), len(y_true)))
    num_correct = 0
    for (x,y) in zip(preds, y_true):
        if x == y:
            num_correct = num_correct + 1
    return num_correct/len(preds)

print("val Accuracy: {}".format(accuracy(val_preds,Y_val)))
print("Predictions evaluated: {}".format(len(val_preds)))

test_preds = mdl.predict(X_test)
print("test Accuracy: {}".format(accuracy(test_preds,Y_test)))


val Accuracy: 0.6085308056872037
Predictions evaluated: 1055
test Accuracy: 0.6009478672985782


In [10]:
# error analysis ##
##################


from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

conf_mat = confusion_matrix(Y_val, preds)
# fig, ax = plt.subplots(figsize=(10,10))
# # sns.heatmap(conf_mat, annot=True, fmt='d',
# #             xticklabels=category_id_df.Product.values, yticklabels=category_id_df.Product.values)
# plt.ylabel('Actual')
# plt.xlabel('Predicted')

# NOTE: this function taken from: 
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = 500
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


plt.figure()
print(conf_mat)
plot_confusion_matrix(conf_mat, classes=mdl.classes_,
                      title="Confusion matrix")
plt.show()

NameError: name 'preds' is not defined

In [ ]:
# test_len = len(preds)
# accuracy = sum(preds)/float(test_len)
# print("accuracy: {}".format(accuracy))

#othello accuracy: 0.7428571428571429. but maybe not permuted
#all: accuracy: 0.328237942884228

#pos, no bleu, 2 plays
# Mult NB: .519

In [ ]:
# doc = nlp(u"a sleep that ends all the heartache")
# # span = doc[doc[4].left_edge.i : doc[4].right_edge.i+1]
# # span.merge()
# for token in doc:
#     print(token.text, token.pos_, token.dep_, token.head.text)

In [ ]:
#raw[:10]

In [ ]:
int(2.1)